In [2]:
import sys
!conda install --yes --prefix {sys.prefix} pandasql

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandasql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py39hf3d152e_0         144 KB  conda-forge
    openssl-1.1.1l             |       h7f98852_0         2.1 MB  conda-forge
    pandasql-0.7.3             |     pyhd8ed1ab_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  pandasql           conda-forge/noarch::pandasql-0.7.3-pyhd8ed1ab_0

The following packages will be UPDATED:

  ca-certificates                      2021.5.30-ha878542_0 --> 2021.10.8-ha878542_0
  certifi                          2021.5.30-

In [6]:
import pandas as p
from pandasql import sqldf
from sklearn.model_selection import train_test_split
pysqldf = lambda q: sqldf(q, globals())

In [4]:
df_people = p.read_csv('../data/People.csv')
df_salaries = p.read_csv('../data/Salaries.csv')
df_appearances = p.read_csv('../data/Appearances.csv')
df_batting = p.read_csv('../data/Batting.csv')
df_fielding = p.read_csv('../data/Fielding.csv')


In [52]:
df_playerids = pysqldf("SELECT DISTINCT playerID from df_people group by playerID")
df_playerids.head(5)

batting_stats = pysqldf("""
    SELECT playerID, yearID, G, (H - BB - SF) / AB as BattingAverage, AB, R, H, HR, RBI, BB, SF 
    FROM df_batting 
    WHERE G >= 50 and AB > 0 and yearID > 1955
""")

player_salaries = pysqldf("""
    SELECT stats.playerID, stats.yearID, df_salaries.salary, G, BattingAverage, AB, R, H, HR, RBI, BB, SF
    FROM batting_stats as stats
    INNER JOIN df_salaries ON df_salaries.playerID = stats.playerID AND df_salaries.yearID = stats.yearID
""")

fielders = pysqldf("""
    SELECT ps.playerID, ps.yearID, salary, POS, ps.G, BattingAverage, AB, R, H, HR, RBI, (PO + A) / (PO + A + E) as FieldingPercentage, A, E, PO
    FROM player_salaries AS ps
    INNER JOIN df_fielding ON ps.playerID=df_fielding.playerID AND ps.yearID=df_fielding.yearID 
""")
fielders.head(5)

,playerID,yearID,salary,POS,G,BattingAverage,AB,R,H,HR,RBI,FieldingPercentage,A,E,PO
0,aguaylu01,1985,237000,2B,91,0.127273,165,27,46,6,21.0,0.981132,25,1.0,27
1,aguaylu01,1985,237000,3B,91,0.127273,165,27,46,6,21.0,1.000000,16,0.0,4
2,aguaylu01,1985,237000,SS,91,0.127273,165,27,46,6,21.0,0.956989,117,8.0,61
3,almonbi01,1985,255000,1B,88,0.168033,244,33,66,6,29.0,0.928571,1,2.0,25
4,almonbi01,1985,255000,3B,88,0.168033,244,33,66,6,29.0,0.857143,4,1.0,2
